In [1]:
# https://spotipy.readthedocs.io/en/2.19.0/

# Install spotipy
# As google colab starts each session like a new computer, we must install this each time.
# If you are working on your local machine, you only need to install it once.
!pip install spotipy --upgrade
!pip install urllib3 --upgrade 

In [2]:
import pandas as pd

## Requests

The requests library is the de facto standard for making HTTP requests in Python. It abstracts the complexities of making requests behind a beautiful, simple API so that you can focus on interacting with services and consuming data in your application.

https://requests.readthedocs.io/en/latest/

When we make a request, we get a requests.Response object in return. The requests.Response object contains the server's response to the HTTP request. Part of this response is a number - which represents whether we received the information we wanted or not. If you get a number that you don't understand, these cats will help you: https://http.cat/

More often than not you'll receive:

200: Success!

401: Unauthorized client error status: lack of valid authentication credentials

403: The server understood the request but refuses to authorize it

In [3]:
import requests

google = requests.get("https://developers.google.com")
print("Google:", google.status_code)

NBA = requests.get("https://api.sportsdata.io/api/nba/fantasy/json/CurrentSeason")
print("NBA:", NBA.status_code) 

wbscs = requests.get("https://www.wbscodingschool.com/")
print("WBS CS: ", wbscs.status_code)

Google: 200
NBA: 401
WBS CS:  403


## JSON

### Intro - making a request and viewing the JSON

https://docs.python.org/3/library/json.html

Since its inception, JSON has quickly become the de facto standard for information exchange. JSON supports primitive types, like strings and numbers, as well as nested lists and objects. It looks like nested python dictionaries:

`{"firstname": "Harry",
"lastname": "Noah",
"city": "Berlin",
"dogs": [{"name": "rover", "breed": "labrador"}, {"name": "pip", "breed": "spaniel"}],
"cars": "none"}`

In [4]:
import json

In [5]:
# Make the request.
response = requests.get("https://jsonplaceholder.typicode.com/users")

In [6]:
# Check the HTTP code.
response

<Response [200]>

In [7]:
# Example of what a JSON looks like.

# View API response as a JSON.
response.json()

[{'id': 1,
  'name': 'Leanne Graham',
  'username': 'Bret',
  'email': 'Sincere@april.biz',
  'address': {'street': 'Kulas Light',
   'suite': 'Apt. 556',
   'city': 'Gwenborough',
   'zipcode': '92998-3874',
   'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
  'phone': '1-770-736-8031 x56442',
  'website': 'hildegard.org',
  'company': {'name': 'Romaguera-Crona',
   'catchPhrase': 'Multi-layered client-server neural-net',
   'bs': 'harness real-time e-markets'}},
 {'id': 2,
  'name': 'Ervin Howell',
  'username': 'Antonette',
  'email': 'Shanna@melissa.tv',
  'address': {'street': 'Victor Plains',
   'suite': 'Suite 879',
   'city': 'Wisokyburgh',
   'zipcode': '90566-7771',
   'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
  'phone': '010-692-6593 x09125',
  'website': 'anastasia.net',
  'company': {'name': 'Deckow-Crist',
   'catchPhrase': 'Proactive didactic contingency',
   'bs': 'synergize scalable supply-chains'}},
 {'id': 3,
  'name': 'Clementine Bauch',
  'username': 'Samantha

### GitHub API - Accessing the data in the JSON

Now that we know
- what an API is,
- how to request information from one (requests),
- how the information will be delivered to us (JSON):

Let's look how we can use this information. We will first look at how we can access particular values within the JSON. Then we will look at a couple of methods to make a dataframe from the JSON.

Github has many APIs. Here we'll look at two of them.

In [8]:
# Very basic API (a string returned).

# GitHub's Zen API produces a new inspirational phrase every 30 seconds.
# Run this cell again in 30 seconds to see a different output.

resp = requests.get("https://api.github.com/zen")
resp.text

'Speak like a human.'

In [9]:
# More complex API (a large json returned).

# Github's Event API shows the events that power the various activity streams on the site.
# In other words, what's happening on Github, who's updating what?

response = requests.get('https://api.github.com/events')
github_response = response.json()
github_response

[{'id': '28470051881',
  'type': 'PullRequestReviewCommentEvent',
  'actor': {'id': 18215579,
   'login': 'CaptainStandby',
   'display_login': 'CaptainStandby',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/CaptainStandby',
   'avatar_url': 'https://avatars.githubusercontent.com/u/18215579?'},
  'repo': {'id': 135321861,
   'name': 'ory/kratos',
   'url': 'https://api.github.com/repos/ory/kratos'},
  'payload': {'action': 'created',
   'comment': {'url': 'https://api.github.com/repos/ory/kratos/pulls/comments/1168962847',
    'pull_request_review_id': 1388468413,
    'id': 1168962847,
    'node_id': 'PRRC_kwDOCBDZBc5FrPUf',
    'diff_hunk': '@@ -198,6 +202,16 @@ func (h *Handler) performNativeLogout(w http.ResponseWriter, r *http.Request, _\n \t\th.d.Writer().WriteError(w, r, err)\n \t\treturn\n \t}\n+\tsess, err := h.d.SessionPersister().GetSessionByToken(r.Context(), p.SessionToken, session.ExpandNothing, identity.ExpandNothing)\n+\tif err != nil {\n+\t\tif errors.Is

In [10]:
# How many events are we looking at?
len(github_response)

30

In [11]:
# What are the keys in the first event?
github_response[0].keys()

dict_keys(['id', 'type', 'actor', 'repo', 'payload', 'public', 'created_at', 'org'])

In [12]:
# We can see that 'repo' is another subdictionary.
# What are the keys in the 'repo' subdictionary?
github_response[0]['repo'].keys()

dict_keys(['id', 'name', 'url'])

In [13]:
# What's the value for the key 'name' in 'repo'?
github_response[0]['repo']['name']

'ory/kratos'

#### Transforming a JSON into a DataFrame

##### Option 1: pd.DataFrame()

In [14]:
# Turn it into a pandas DataFrame.
pd.DataFrame(github_response)

,id,type,actor,repo,payload,public,created_at,org
0,28470051881,PullRequestReviewCommentEvent,"{'id': 18215579, 'login': 'CaptainStandby', 'd...","{'id': 135321861, 'name': 'ory/kratos', 'url':...","{'action': 'created', 'comment': {'url': 'http...",True,2023-04-17T16:03:39Z,"{'id': 25334553, 'login': 'ory', 'gravatar_id'..."
1,28470051889,MemberEvent,"{'id': 117729152, 'login': 'Xernnn', 'display_...","{'id': 621684127, 'name': 'Xernnn/Comics', 'ur...","{'member': {'login': 'anminhtri', 'id': 100016...",True,2023-04-17T16:07:55Z,NaN
2,28470051831,PushEvent,"{'id': 13872904, 'login': 'KaushikRishabh', 'd...","{'id': 629100603, 'name': 'KaushikRishabh/reac...","{'repository_id': 629100603, 'push_id': 133289...",True,2023-04-17T16:07:55Z,NaN
3,28470051974,CreateEvent,"{'id': 119161971, 'login': 'ajeetdp', 'display...","{'id': 629100459, 'name': 'ajeetdp/cricketworl...","{'ref': 'main', 'ref_type': 'branch', 'master_...",True,2023-04-17T16:07:55Z,NaN
4,28470051859,PushEvent,"{'id': 34553908, 'login': 'dschibster', 'displ...","{'id': 586995654, 'name': 'mind-labs/sf-dynami...","{'repository_id': 586995654, 'push_id': 133289...",True,2023-04-17T16:07:55Z,"{'id': 122306330, 'login': 'mind-labs', 'grava..."
5,28470051900,CreateEvent,"{'id': 75875444, 'login': 'hitenanand97', 'dis...","{'id': 629100613, 'name': 'hitenanand97/hitena...","{'ref': 'main', 'ref_type': 'branch', 'master_...",True,2023-04-17T16:07:55Z,NaN
6,28470051910,PushEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 594169901, 'name': 'peter279k/cdc-data-...","{'repository_id': 594169901, 'push_id': 133289...",True,2023-04-17T16:07:55Z,NaN
7,28470051935,PushEvent,"{'id': 76601914, 'login': 'fullfeeds', 'displa...","{'id': 324450602, 'name': 'fullfeeds/feeds', '...","{'repository_id': 324450602, 'push_id': 133289...",True,2023-04-17T16:07:55Z,NaN
8,28470051855,PushEvent,"{'id': 129499031, 'login': 'Christoffel-T', 'd...","{'id': 621725696, 'name': 'Christoffel-T/fiver...","{'repository_id': 621725696, 'push_id': 133289...",True,2023-04-17T16:07:55Z,NaN
9,28470051893,PullRequestEvent,"{'id': 45748163, 'login': 'Averagess', 'displa...","{'id': 592097254, 'name': 'Averagess/bitcollec...","{'action': 'opened', 'number': 62, 'pull_reque...",True,2023-04-17T16:07:55Z,NaN


##### Option 2: pd.json_normalize()
You may notice above that many columns still contain dictionaries as values. We can correct this using json_normalize().

https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html

In [15]:
pd.json_normalize(github_response)

,id,type,public,created_at,actor.id,actor.login,actor.display_login,actor.gravatar_id,actor.url,actor.avatar_url,...,payload.issue.state_reason,payload.comment.issue_url,payload.comment.performed_via_github_app,payload.comment.in_reply_to_id,payload.issue.draft,payload.issue.pull_request.url,payload.issue.pull_request.html_url,payload.issue.pull_request.diff_url,payload.issue.pull_request.patch_url,payload.issue.pull_request.merged_at
0,28470051881,PullRequestReviewCommentEvent,True,2023-04-17T16:03:39Z,18215579,CaptainStandby,CaptainStandby,,https://api.github.com/users/CaptainStandby,https://avatars.githubusercontent.com/u/18215579?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28470051889,MemberEvent,True,2023-04-17T16:07:55Z,117729152,Xernnn,Xernnn,,https://api.github.com/users/Xernnn,https://avatars.githubusercontent.com/u/117729...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28470051831,PushEvent,True,2023-04-17T16:07:55Z,13872904,KaushikRishabh,KaushikRishabh,,https://api.github.com/users/KaushikRishabh,https://avatars.githubusercontent.com/u/13872904?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28470051974,CreateEvent,True,2023-04-17T16:07:55Z,119161971,ajeetdp,ajeetdp,,https://api.github.com/users/ajeetdp,https://avatars.githubusercontent.com/u/119161...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28470051859,PushEvent,True,2023-04-17T16:07:55Z,34553908,dschibster,dschibster,,https://api.github.com/users/dschibster,https://avatars.githubusercontent.com/u/34553908?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,28470051900,CreateEvent,True,2023-04-17T16:07:55Z,75875444,hitenanand97,hitenanand97,,https://api.github.com/users/hitenanand97,https://avatars.githubusercontent.com/u/75875444?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,28470051910,PushEvent,True,2023-04-17T16:07:55Z,41898282,github-actions[bot],github-actions,,https://api.github.com/users/github-actions[bot],https://avatars.githubusercontent.com/u/41898282?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,28470051935,PushEvent,True,2023-04-17T16:07:55Z,76601914,fullfeeds,fullfeeds,,https://api.github.com/users/fullfeeds,https://avatars.githubusercontent.com/u/76601914?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,28470051855,PushEvent,True,2023-04-17T16:07:55Z,129499031,Christoffel-T,Christoffel-T,,https://api.github.com/users/Christoffel-T,https://avatars.githubusercontent.com/u/129499...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,28470051893,PullRequestEvent,True,2023-04-17T16:07:55Z,45748163,Averagess,Averagess,,https://api.github.com/users/Averagess,https://avatars.githubusercontent.com/u/45748163?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Selecting only certain values by iterating with a for loop

If we only want to select certain parts of the JSON:
- Option 1: make a DataFrame and drop the rest.
- Option 2: Use a for loop to extract only the required information.

https://www.w3schools.com/python/python_for_loops.asp

https://www.w3schools.com/python/ref_func_range.asp

In [16]:
# login - first value
github_response[0]['actor']['login']

'CaptainStandby'

In [17]:
# repo - first value
github_response[0]['repo']['name']

'ory/kratos'

In [18]:
# event_type - first value
github_response[0]['type']

'PullRequestReviewCommentEvent'

In [19]:
# Empty lists that the loop will fill with values.
login = []
repo = []
event_type = []

for i in range(len(github_response)):
    # add the ith login value to the login list
    login.append(github_response[i]['actor']['login'])
    # add the ith repo name to the repo list
    repo.append(github_response[i]['repo']['name'])
    # add the ith event type to the event_type list
    event_type.append(github_response[i]['type'])

In [20]:
# Let's have a look at the login list.
login

['CaptainStandby',
 'Xernnn',
 'KaushikRishabh',
 'ajeetdp',
 'dschibster',
 'hitenanand97',
 'github-actions[bot]',
 'fullfeeds',
 'Christoffel-T',
 'Averagess',
 'egorps4',
 'oss-fuzz-robot',
 'github-actions[bot]',
 'github-actions[bot]',
 'AidanCox122',
 'cston',
 'akosma',
 'vercel[bot]',
 'DarkOffical',
 'hellblazer7',
 'GungIndi',
 'tbjump',
 'dependabot[bot]',
 'CaptainStandby',
 'Boygame666',
 'AmirJlr',
 'renovate-bot',
 'dependabot[bot]',
 'LSB616',
 'nickchapman-da']

In [21]:
# Let's have a look at the repo list.
repo

['ory/kratos',
 'Xernnn/Comics',
 'KaushikRishabh/react-html-to-rich-text',
 'ajeetdp/cricketworld',
 'mind-labs/sf-dynamic-approvals',
 'hitenanand97/hitenanand97',
 'peter279k/cdc-data-mirror',
 'fullfeeds/feeds',
 'Christoffel-T/fiverr-pat-20230331',
 'Averagess/bitcollector-bot',
 'egorps4/XBOX',
 'systemd/systemd',
 'prakashtv007/UTLive2',
 'kentayamada-dev/kentayamada-dev',
 'AidanCox122/blues-makos',
 'cston/roslyn',
 'vshn/nginx',
 'igornevesg/DR-Paulo',
 'DarkOffical/Elsa',
 'hellblazer7/Moody-Much',
 'GungIndi/Simple-Online-Restaurant',
 'wormhole-foundation/wormhole',
 'spectrocloud/cluster-api-provider-aws',
 'ory/kratos',
 'Boygame666/GoodGreen',
 'LisaHJung/Beginners-Crash-Course-to-Elastic-Stack-Series-Table-of-Contents',
 'renovate-bot/common-protos-ruby',
 'libbpf/libbpf-rs',
 'LSB616/music_app',
 'digital-asset/daml']

In [22]:
# Let's have a look at the event_type list.
event_type

['PullRequestReviewCommentEvent',
 'MemberEvent',
 'PushEvent',
 'CreateEvent',
 'PushEvent',
 'CreateEvent',
 'PushEvent',
 'PushEvent',
 'PushEvent',
 'PullRequestEvent',
 'PushEvent',
 'IssueCommentEvent',
 'PushEvent',
 'PushEvent',
 'PushEvent',
 'PushEvent',
 'DeleteEvent',
 'CommitCommentEvent',
 'PushEvent',
 'PushEvent',
 'PublicEvent',
 'PullRequestReviewCommentEvent',
 'IssueCommentEvent',
 'PullRequestReviewCommentEvent',
 'PushEvent',
 'WatchEvent',
 'PushEvent',
 'DeleteEvent',
 'PushEvent',
 'PushEvent']

### International Space Station API - just another cool API

Send a simple `get` request to know where the ISS is right now.

Docs here: http://open-notify.org/Open-Notify-API/ISS-Location-Now/

In [23]:
url = "http://api.open-notify.org/iss-now.json"

In [24]:
response = requests.get(url)

In [25]:
response.json()

{'message': 'success',
 'timestamp': 1681747997,
 'iss_position': {'latitude': '31.1766', 'longitude': '-29.0851'}}

## Spotipy

Now that you know all about APIs, let's use that knowledge on something even more fun.

Spotify has an API that allows users to gather information about songs and even interact with other users and playlists. To make their usage in Python easier, someone created `spotipy`, a library with some convenient functions to send requests and collect data.

Create / log into a Spotify account (https://developer.spotify.com/dashboard/login) and follow these steps (only the "Register your App" section): https://developer.spotify.com/documentation/general/guides/authorization/app-settings/

#### Authentication

With most APIs we need to autheticate ourself. This is often done with a username and a password. You will likely use a different username and password for most APIs, so make sure you're using a password manager, or keep everything written down somewhere safe.

If ever you'd like more information about spotipy: [here are the docs](https://spotipy.readthedocs.io/en/2.19.0/).

In [26]:
# import libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [27]:
#Initialize SpotiPy with user credentials
#sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    #client_id="82d10cbeab244ba885a29af43becd014",
    #client_secret="b23ecbf78eb743b38161dae226870f4c"))


In [32]:
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id="e5663f97f8484147b4e34ffc56894d91", # 
    client_secret="f770102854264b40b26c8cf54022ab3b")) # 

#### Searching songs with 'queries' with `sp.search`

This method allows you to find songs using Spotify's search engine. That's convenient when you don't have the exact "id" of a song.

In [33]:
# Search for 'Lady Gaga', restricted to the first 10 results.

results = sp.search(q="Lady Gaga", limit = 10)

Explore the object returned by the request. As it's a dictionary (with nested dictionaries inside), using `.keys` is a great way to see what's in there:

In [34]:
results.keys()

dict_keys(['tracks'])

In [35]:
# Only one key makes it seem like there'll be many subdictionaries, let's delve deeper.
results["tracks"].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

This is the url of your request:

In [36]:
# You can play the track by pasting the url in your browser.

# We can explore further by adding keys one after the other.

results["tracks"]["href"]

'https://api.spotify.com/v1/search?query=Lady+Gaga&type=track&offset=0&limit=10'

This is the name of the first song returned by the API:

In [37]:
results["tracks"]["items"][0]["name"]

'Poker Face'

As one song can have many artists, the artists are returned as a list: note the square brackets.

In [38]:
results["tracks"]["items"][0]["artists"]

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1HY2Jd0NmPuamShAr6KMms'},
  'href': 'https://api.spotify.com/v1/artists/1HY2Jd0NmPuamShAr6KMms',
  'id': '1HY2Jd0NmPuamShAr6KMms',
  'name': 'Lady Gaga',
  'type': 'artist',
  'uri': 'spotify:artist:1HY2Jd0NmPuamShAr6KMms'}]

There are some other interesting features contained in the search results:

In [39]:
# https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-tracks
# The popularity of the track. The value will be between 0 and 100, with 100 being the most popular.

results["tracks"]["items"][0]["popularity"]

78

This is how Spotify identifies individual songs: with a Uniform Resource Identifier  or `uri`. (The `id` and the `url` are also ways to identify each song uniquely).

In [40]:
results["tracks"]["items"][0]["uri"]

'spotify:track:5R8dQOPq8haW94K7mgERlO'

Here we look for 10 songs by the Red Hot Chili Peppers and store the `uri` of the songs and their names.

In [41]:
# Send request and store the response.
red_hot = sp.search(q="Red Hot Chili Peppers", limit=10)

# Initialize empty lists that we will fill with information from our loop.
list_of_uri = []
list_of_song_names = []

# Iterate through the "items" (the songs),
# and append the "uri" and the "name" to the lists we created.
for item in red_hot["tracks"]["items"]:
    list_of_uri.append(item["uri"])
    list_of_song_names.append(item["name"])

# Print results.
print(list_of_uri)
print("\n")
print(list_of_song_names)

['spotify:track:3d9DChrdc6BOeFsbrZ3Is0', 'spotify:track:0uppYCG86ajpV2hSR3dJJ0', 'spotify:track:64BbK9SFKH2jk86U3dGj2P', 'spotify:track:1G391cbiT3v3Cywg8T7DM1', 'spotify:track:48UPSzbZjgc449aqz8bxox', 'spotify:track:3ZOEytgrvLwQaqXreDs2Jx', 'spotify:track:2aibwv5hGXSgw7Yru8IYTO', 'spotify:track:10Nmj3JCNoMeBQ87uw5j8k', 'spotify:track:1ndGB6rvxKYN9seCYO1dTF', 'spotify:track:1f2V8U1BiWaC9aJWmpOARe']


['Under the Bridge', 'Give It Away', 'Otherside', 'Scar Tissue', 'Californication', "Can't Stop", 'Snow (Hey Oh)', 'Dani California', 'The Zephyr Song', 'By the Way']


#### Searching multiple artists

Here we first create a list of artists we want to gather songs from. Then we iterate through them and append the results to a big list called `results`.

In [42]:
artists = ["Red Hot Chili Peppers", "SCARR", "Whitney Houston"]

In [43]:
results = []

for artist in artists:
    results.append(sp.search(q=artist, limit=10)) 

In [44]:
# Let's look at the second element in the results list.
results[1]

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=SCARR&type=track&offset=0&limit=10',
  'items': [{'album': {'album_group': 'album',
     'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4fxd5Ee7UefO4CUXgwJ7IP'},
       'href': 'https://api.spotify.com/v1/artists/4fxd5Ee7UefO4CUXgwJ7IP',
       'id': '4fxd5Ee7UefO4CUXgwJ7IP',
       'name': 'Giveon',
       'type': 'artist',
       'uri': 'spotify:artist:4fxd5Ee7UefO4CUXgwJ7IP'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ'

We can iterate through the `results` list and get just the names of all the songs:

In [45]:
song_names = []

for result in results:
    for item in result["tracks"]["items"]:
        song_names.append(item["name"])

In [46]:
song_names

['Under the Bridge',
 'Give It Away',
 'Otherside',
 'Scar Tissue',
 'Californication',
 "Can't Stop",
 'Snow (Hey Oh)',
 'Dani California',
 'The Zephyr Song',
 'By the Way',
 'Scarred',
 'SCARR',
 'scar',
 'Scarred From Love',
 'SCARS',
 'Scarred from Love',
 'Scar',
 "Scarred Baby's",
 'SCAR',
 'Scarred',
 'I Wanna Dance with Somebody (Who Loves Me)',
 'I Will Always Love You',
 'Higher Love',
 'I Have Nothing',
 "It's Not Right But It's Okay",
 'Saving All My Love for You',
 'How Will I Know',
 'Greatest Love of All',
 'Higher Love',
 'How Will I Know']

### Playlists

Using spotipy, we can both build and read spotify playlists. Today, we will only show you how to read information from a playlist. However, if you wish to build one, we strongly encourage you read the [documentation](https://spotipy.readthedocs.io/en/2.19.0/) and explore further.

In [47]:
my_playlist = sp.user_playlist_tracks(user="spotify", playlist_id="spotify:playlist:0ce6Rmxf7QXroqa1wzjWY8")

Extract songs ID from a playlist

In [48]:
my_playlist

{'href': 'https://api.spotify.com/v1/playlists/0ce6Rmxf7QXroqa1wzjWY8/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2021-09-22T07:34:05Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/gperdigo'},
    'href': 'https://api.spotify.com/v1/users/gperdigo',
    'id': 'gperdigo',
    'type': 'user',
    'uri': 'spotify:user:gperdigo'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_group': 'single',
     'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/579T5fNgfbDetyamUTAetw'},
       'href': 'https://api.spotify.com/v1/artists/579T5fNgfbDetyamUTAetw',
       'id': '579T5fNgfbDetyamUTAetw',
       'name': 'S+C+A+R+R',
       'type': 'artist',
       'uri': 'spotify:artist:579T5fNgfbDetyamUTAetw'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA'

In [49]:
my_playlist["items"][0]["track"]["uri"]

'spotify:track:5Tnx4R7Gwj1LZsfssfzchh'

### Audio features

You can check an explanation of the audio features [here](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/).

In [50]:
sp.audio_features("spotify:track:6Sy9BUbgFse0n0LPA5lwy5")

[{'danceability': 0.528,
  'energy': 0.965,
  'key': 11,
  'loudness': -7.984,
  'mode': 0,
  'speechiness': 0.0465,
  'acousticness': 0.141,
  'instrumentalness': 0.985,
  'liveness': 0.0797,
  'valence': 0.587,
  'tempo': 136.065,
  'type': 'audio_features',
  'id': '6Sy9BUbgFse0n0LPA5lwy5',
  'uri': 'spotify:track:6Sy9BUbgFse0n0LPA5lwy5',
  'track_href': 'https://api.spotify.com/v1/tracks/6Sy9BUbgFse0n0LPA5lwy5',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6Sy9BUbgFse0n0LPA5lwy5',
  'duration_ms': 225493,
  'time_signature': 4}]

### Creating a function that takes a song name and returns its audio features 

In [51]:
list_of_songs = []

def song_features(human_song_title):
    # Search for the song title you enter into the function, limited to the first 10 results.
    results = sp.search(q=human_song_title, limit = 10)['tracks']['items']
    
    # Create a loop, so we only select the parts of the json we need.
    for i in results:

        # Empty dictionary to be filled with the information below.
        track_dict = {}

        # Add the key artist and a corresponding value to the dictionary.
        track_dict['Artist'] = i['artists'][0]['name']
        # Add the key title and the corresponding value to the dictionary.
        track_dict['Title'] = i['name'] 
        # Add the key album and the corresponding value to the dictionary.
        track_dict['Album'] = i['album']['name']
        # Add the key audio description and the corresponding value to the dictionary.
        track_dict['Audio Description'] = sp.audio_features(i['id'])
        
        # Add the dictionary to the list list_of_songs.
        list_of_songs.append(track_dict)
    # Output list_of_songs.
    return list_of_songs

# Call the function with a song to test.
song_features("Under the Bridge")

[{'Artist': 'Red Hot Chili Peppers',
  'Title': 'Under the Bridge',
  'Album': 'Blood Sugar Sex Magik (Deluxe Edition)',
  'Audio Description': [{'danceability': 0.559,
    'energy': 0.345,
    'key': 4,
    'loudness': -13.496,
    'mode': 1,
    'speechiness': 0.0459,
    'acousticness': 0.0576,
    'instrumentalness': 0.000105,
    'liveness': 0.141,
    'valence': 0.458,
    'tempo': 84.581,
    'type': 'audio_features',
    'id': '3d9DChrdc6BOeFsbrZ3Is0',
    'uri': 'spotify:track:3d9DChrdc6BOeFsbrZ3Is0',
    'track_href': 'https://api.spotify.com/v1/tracks/3d9DChrdc6BOeFsbrZ3Is0',
    'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3d9DChrdc6BOeFsbrZ3Is0',
    'duration_ms': 264307,
    'time_signature': 4}]},
 {'Artist': 'owlh',
  'Title': 'Under the Bridge',
  'Album': 'cover to cover',
  'Audio Description': [{'danceability': 0.781,
    'energy': 0.355,
    'key': 9,
    'loudness': -8.158,
    'mode': 1,
    'speechiness': 0.0376,
    'acousticness': 0.471,
    'in

In [52]:
# Make a dataframe from the list of songs created in the function above.
df = pd.DataFrame(list_of_songs)

df

,Artist,Title,Album,Audio Description
0,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik (Deluxe Edition),"[{'danceability': 0.559, 'energy': 0.345, 'key..."
1,owlh,Under the Bridge,cover to cover,"[{'danceability': 0.781, 'energy': 0.355, 'key..."
2,Red Hot Chili Peppers,Under the Bridge,Greatest Hits,"[{'danceability': 0.554, 'energy': 0.49, 'key'..."
3,Adele,Water Under the Bridge,25,"[{'danceability': 0.59, 'energy': 0.833, 'key'..."
4,Sam Hunt,Water Under The Bridge,Water Under The Bridge,"[{'danceability': 0.635, 'energy': 0.866, 'key..."
5,Rockabye Baby!,Under the Bridge,Lullaby Renditions of Red Hot Chili Peppers,"[{'danceability': 0.704, 'energy': 0.06, 'key'..."
6,John Altman,Under The Bridges Of Paris,Shall We Dance?,"[{'danceability': 0.204, 'energy': 0.376, 'key..."
7,The Drifters,Under the Boardwalk,Under the Boardwalk,"[{'danceability': 0.747, 'energy': 0.226, 'key..."
8,All Saints,Under the Bridge,All Saints,"[{'danceability': 0.717, 'energy': 0.564, 'key..."
9,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik,"[{'danceability': 0.559, 'energy': 0.345, 'key..."


As you can see, this DataFrame looks a bit off as the audio descriptions aren't expanded - all of the data is clumped together in one cell. Let's correct this, so we can see each audio feature as an individual column.

In [53]:
# Quick function we can use to select only the first item.
# This can also be done simply with [0], but we wanted to show you how you can incorporate a custom function into your work.

def first_value (x):
    return x[0]

# Making a DataFrame from the audio features of the songs in list_of_songs.
df_audio_features = pd.json_normalize(df['Audio Description'].apply(first_value))

df_audio_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.559,0.345,4,-13.496,1,0.0459,0.0576,0.000105,0.1410,0.458,84.581,audio_features,3d9DChrdc6BOeFsbrZ3Is0,spotify:track:3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,https://api.spotify.com/v1/audio-analysis/3d9D...,264307,4
1,0.781,0.355,9,-8.158,1,0.0376,0.4710,0.878000,0.1250,0.363,131.978,audio_features,1AyRpRvLwUwkXwGGuNZHmN,spotify:track:1AyRpRvLwUwkXwGGuNZHmN,https://api.spotify.com/v1/tracks/1AyRpRvLwUwk...,https://api.spotify.com/v1/audio-analysis/1AyR...,244196,4
2,0.554,0.490,4,-8.046,1,0.0457,0.0168,0.000534,0.1360,0.513,84.275,audio_features,23NPGXlSaIqWzvxIRhM2oG,spotify:track:23NPGXlSaIqWzvxIRhM2oG,https://api.spotify.com/v1/tracks/23NPGXlSaIqW...,https://api.spotify.com/v1/audio-analysis/23NP...,265507,4
3,0.590,0.833,5,-6.503,0,0.0615,0.0142,0.000005,0.1050,0.538,94.963,audio_features,4jL6WWKFDqCOPo2hC3VhSS,spotify:track:4jL6WWKFDqCOPo2hC3VhSS,https://api.spotify.com/v1/tracks/4jL6WWKFDqCO...,https://api.spotify.com/v1/audio-analysis/4jL6...,240439,4
4,0.635,0.866,5,-2.695,1,0.0473,0.0697,0.000000,0.0955,0.853,133.889,audio_features,4viFAHmivkYQKkwLvwKOgg,spotify:track:4viFAHmivkYQKkwLvwKOgg,https://api.spotify.com/v1/tracks/4viFAHmivkYQ...,https://api.spotify.com/v1/audio-analysis/4viF...,169760,4
5,0.704,0.060,4,-21.087,1,0.0907,0.9460,0.912000,0.1090,0.791,158.085,audio_features,45VCZtguiCNu5zJ3YkmcWC,spotify:track:45VCZtguiCNu5zJ3YkmcWC,https://api.spotify.com/v1/tracks/45VCZtguiCNu...,https://api.spotify.com/v1/audio-analysis/45VC...,259693,4
6,0.204,0.376,5,-11.476,1,0.0345,0.7580,0.496000,0.0931,0.235,93.075,audio_features,6K7jgMdtOVGQVqHbGAgRuM,spotify:track:6K7jgMdtOVGQVqHbGAgRuM,https://api.spotify.com/v1/tracks/6K7jgMdtOVGQ...,https://api.spotify.com/v1/audio-analysis/6K7j...,180613,3
7,0.747,0.226,7,-13.635,1,0.0414,0.9080,0.000168,0.1070,0.778,123.792,audio_features,65jrjEhWfAvysKfnojk1i0,spotify:track:65jrjEhWfAvysKfnojk1i0,https://api.spotify.com/v1/tracks/65jrjEhWfAvy...,https://api.spotify.com/v1/audio-analysis/65jr...,161960,4
8,0.717,0.564,6,-7.752,0,0.0551,0.0452,0.000464,0.0897,0.728,85.996,audio_features,78M2x5ojbdTmUOpWrbijG3,spotify:track:78M2x5ojbdTmUOpWrbijG3,https://api.spotify.com/v1/tracks/78M2x5ojbdTm...,https://api.spotify.com/v1/audio-analysis/78M2...,301067,4
9,0.559,0.345,4,-13.496,1,0.0459,0.0576,0.000105,0.1410,0.458,84.581,audio_features,5PclxRY6shIQzSKxxbdZso,spotify:track:5PclxRY6shIQzSKxxbdZso,https://api.spotify.com/v1/tracks/5PclxRY6shIQ...,https://api.spotify.com/v1/audio-analysis/5Pcl...,264307,4


In [54]:
# Merge the expanded audio features with the original DataFrame.
new_df = pd.merge(df, df_audio_features, left_index=True, right_index=True)

# Drop the old ugly column where all the audio features are clumped together.
new_df.drop('Audio Description', axis=1, inplace=True)

new_df

,Artist,Title,Album,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik (Deluxe Edition),0.559,0.345,4,-13.496,1,0.0459,0.0576,...,0.1410,0.458,84.581,audio_features,3d9DChrdc6BOeFsbrZ3Is0,spotify:track:3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,https://api.spotify.com/v1/audio-analysis/3d9D...,264307,4
1,owlh,Under the Bridge,cover to cover,0.781,0.355,9,-8.158,1,0.0376,0.4710,...,0.1250,0.363,131.978,audio_features,1AyRpRvLwUwkXwGGuNZHmN,spotify:track:1AyRpRvLwUwkXwGGuNZHmN,https://api.spotify.com/v1/tracks/1AyRpRvLwUwk...,https://api.spotify.com/v1/audio-analysis/1AyR...,244196,4
2,Red Hot Chili Peppers,Under the Bridge,Greatest Hits,0.554,0.490,4,-8.046,1,0.0457,0.0168,...,0.1360,0.513,84.275,audio_features,23NPGXlSaIqWzvxIRhM2oG,spotify:track:23NPGXlSaIqWzvxIRhM2oG,https://api.spotify.com/v1/tracks/23NPGXlSaIqW...,https://api.spotify.com/v1/audio-analysis/23NP...,265507,4
3,Adele,Water Under the Bridge,25,0.590,0.833,5,-6.503,0,0.0615,0.0142,...,0.1050,0.538,94.963,audio_features,4jL6WWKFDqCOPo2hC3VhSS,spotify:track:4jL6WWKFDqCOPo2hC3VhSS,https://api.spotify.com/v1/tracks/4jL6WWKFDqCO...,https://api.spotify.com/v1/audio-analysis/4jL6...,240439,4
4,Sam Hunt,Water Under The Bridge,Water Under The Bridge,0.635,0.866,5,-2.695,1,0.0473,0.0697,...,0.0955,0.853,133.889,audio_features,4viFAHmivkYQKkwLvwKOgg,spotify:track:4viFAHmivkYQKkwLvwKOgg,https://api.spotify.com/v1/tracks/4viFAHmivkYQ...,https://api.spotify.com/v1/audio-analysis/4viF...,169760,4
5,Rockabye Baby!,Under the Bridge,Lullaby Renditions of Red Hot Chili Peppers,0.704,0.060,4,-21.087,1,0.0907,0.9460,...,0.1090,0.791,158.085,audio_features,45VCZtguiCNu5zJ3YkmcWC,spotify:track:45VCZtguiCNu5zJ3YkmcWC,https://api.spotify.com/v1/tracks/45VCZtguiCNu...,https://api.spotify.com/v1/audio-analysis/45VC...,259693,4
6,John Altman,Under The Bridges Of Paris,Shall We Dance?,0.204,0.376,5,-11.476,1,0.0345,0.7580,...,0.0931,0.235,93.075,audio_features,6K7jgMdtOVGQVqHbGAgRuM,spotify:track:6K7jgMdtOVGQVqHbGAgRuM,https://api.spotify.com/v1/tracks/6K7jgMdtOVGQ...,https://api.spotify.com/v1/audio-analysis/6K7j...,180613,3
7,The Drifters,Under the Boardwalk,Under the Boardwalk,0.747,0.226,7,-13.635,1,0.0414,0.9080,...,0.1070,0.778,123.792,audio_features,65jrjEhWfAvysKfnojk1i0,spotify:track:65jrjEhWfAvysKfnojk1i0,https://api.spotify.com/v1/tracks/65jrjEhWfAvy...,https://api.spotify.com/v1/audio-analysis/65jr...,161960,4
8,All Saints,Under the Bridge,All Saints,0.717,0.564,6,-7.752,0,0.0551,0.0452,...,0.0897,0.728,85.996,audio_features,78M2x5ojbdTmUOpWrbijG3,spotify:track:78M2x5ojbdTmUOpWrbijG3,https://api.spotify.com/v1/tracks/78M2x5ojbdTm...,https://api.spotify.com/v1/audio-analysis/78M2...,301067,4
9,Red Hot Chili Peppers,Under the Bridge,Blood Sugar Sex Magik,0.559,0.345,4,-13.496,1,0.0459,0.0576,...,0.1410,0.458,84.581,audio_features,5PclxRY6shIQzSKxxbdZso,spotify:track:5PclxRY6shIQzSKxxbdZso,https://api.spotify.com/v1/tracks/5PclxRY6shIQ...,https://api.spotify.com/v1/audio-analysis/5Pcl...,264307,4


If you like a challenge and want to have a go at testing what you've just learned: Follow the steps below and see if you can make a DataFrame similar to the one above with expanded audio features. This time though, do it for a playlist of your choice on spotify. Then see if you can expand it to include the songs from multiple playlists.

Try not to look at the solution, but we've included it below in case you get stuck

### Collect a big dataframe of songs with their audio features

- Start by looking for a playlist on spotify (it does not have to be your playlist), and copy its url.

- Extract the audio features for each song on your playlist.

- Collect the link of many playlists and do the same for all of them.

- Structure the information as a dataframe where each row is a song and the columns are audio features.

#### Solution

In [61]:
list_of_playlists = ["spotify:playlist:2VHdhD77ZP0A3GuJ1MVEfQ",
                    "spotify:playlist:0ce6Rmxf7QXroqa1wzjWY8"]

track_list = []
for i in list_of_playlists:
    individual_playlist = sp.user_playlist_tracks(user="spotify", playlist_id=i)['items']
    for j in individual_playlist:
        track_dict = {}
        track_dict['Artist'] = j['track']['artists'][0]['name']
        track_dict['Title'] = j['track']['name']
        track_dict['Album'] = j['track']['album']['name']
        track_dict['Audio Description'] = sp.audio_features(j['track']['id'])
        track_list.append(track_dict)

print(track_list)

[{'Artist': 'Zack Knight', 'Title': 'Bom Diggy Diggy', 'Album': 'Bom Diggy Diggy (From "Sonu Ke Titu Ki Sweety")', 'Audio Description': [{'danceability': 0.782, 'energy': 0.827, 'key': 11, 'loudness': -4.219, 'mode': 0, 'speechiness': 0.0627, 'acousticness': 0.159, 'instrumentalness': 0, 'liveness': 0.226, 'valence': 0.498, 'tempo': 104.032, 'type': 'audio_features', 'id': '6qCNaRRr5xJALfNJvh0NAw', 'uri': 'spotify:track:6qCNaRRr5xJALfNJvh0NAw', 'track_href': 'https://api.spotify.com/v1/tracks/6qCNaRRr5xJALfNJvh0NAw', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6qCNaRRr5xJALfNJvh0NAw', 'duration_ms': 238530, 'time_signature': 4}]}, {'Artist': 'Neha Kakkar', 'Title': 'Dilbar', 'Album': 'Satyameva Jayate', 'Audio Description': [{'danceability': 0.725, 'energy': 0.912, 'key': 9, 'loudness': -3.665, 'mode': 0, 'speechiness': 0.0851, 'acousticness': 0.155, 'instrumentalness': 7.68e-05, 'liveness': 0.107, 'valence': 0.674, 'tempo': 104.054, 'type': 'audio_features', 'id': '0Ms1

In [62]:
playlist_df = pd.DataFrame(track_list)

In [63]:
df_a_f = pd.json_normalize(playlist_df['Audio Description'].apply(first_value))

In [64]:
new_playlist_df = pd.merge(playlist_df, df_a_f, left_index=True, right_index=True)

In [65]:
new_playlist_df.drop('Audio Description', axis=1, inplace=True)

In [66]:
new_playlist_df

,Artist,Title,Album,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Zack Knight,Bom Diggy Diggy,"Bom Diggy Diggy (From ""Sonu Ke Titu Ki Sweety"")",0.782,0.827,11,-4.219,0,0.0627,0.1590,...,0.2260,0.498,104.032,audio_features,6qCNaRRr5xJALfNJvh0NAw,spotify:track:6qCNaRRr5xJALfNJvh0NAw,https://api.spotify.com/v1/tracks/6qCNaRRr5xJA...,https://api.spotify.com/v1/audio-analysis/6qCN...,238530,4
1,Neha Kakkar,Dilbar,Satyameva Jayate,0.725,0.912,9,-3.665,0,0.0851,0.1550,...,0.1070,0.674,104.054,audio_features,0Ms1V2flsPzr2bVqImelCB,spotify:track:0Ms1V2flsPzr2bVqImelCB,https://api.spotify.com/v1/tracks/0Ms1V2flsPzr...,https://api.spotify.com/v1/audio-analysis/0Ms1...,184433,4
2,Zack Knight,Thumka,Thumka,0.778,0.801,5,-6.456,1,0.0758,0.0284,...,0.3270,0.620,106.061,audio_features,7sw49gd22TmoSW3sMGuHbn,spotify:track:7sw49gd22TmoSW3sMGuHbn,https://api.spotify.com/v1/tracks/7sw49gd22Tmo...,https://api.spotify.com/v1/audio-analysis/7sw4...,167438,4
3,Shreya Ghoshal,"Slow Motion (From ""Bharat"")","Slow Motion (From ""Bharat"")",0.574,0.738,10,-5.526,1,0.0958,0.0235,...,0.0959,0.307,109.996,audio_features,7yEuRloGyxyu1KaWBTPP1Z,spotify:track:7yEuRloGyxyu1KaWBTPP1Z,https://api.spotify.com/v1/tracks/7yEuRloGyxyu...,https://api.spotify.com/v1/audio-analysis/7yEu...,247248,4
4,Aastha Gill,Kamariya,Stree,0.825,0.666,4,-4.847,0,0.0554,0.1430,...,0.2370,0.763,96.987,audio_features,5cjVsWqIkBQC7acTRhL0RO,spotify:track:5cjVsWqIkBQC7acTRhL0RO,https://api.spotify.com/v1/tracks/5cjVsWqIkBQC...,https://api.spotify.com/v1/audio-analysis/5cjV...,187983,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,Tiësto,The Business,The Business,0.798,0.620,8,-7.079,0,0.2320,0.4140,...,0.1120,0.235,120.031,audio_features,6f3Slt0GbA2bPZlz0aIFXN,spotify:track:6f3Slt0GbA2bPZlz0aIFXN,https://api.spotify.com/v1/tracks/6f3Slt0GbA2b...,https://api.spotify.com/v1/audio-analysis/6f3S...,164000,4
112,Johnny Hooker,Amor Marginal,"Eu Vou Fazer uma Macumba Pra Te Amarrar, Maldito!",0.423,0.539,9,-5.722,0,0.0368,0.2520,...,0.0959,0.196,170.510,audio_features,7uSdpAkxgB0948bW7yjjJO,spotify:track:7uSdpAkxgB0948bW7yjjJO,https://api.spotify.com/v1/tracks/7uSdpAkxgB09...,https://api.spotify.com/v1/audio-analysis/7uSd...,285000,3
113,The Tragically Hip,Ahead By A Century,Yer Favourites,0.778,0.628,7,-7.306,1,0.0275,0.0164,...,0.0862,0.356,126.586,audio_features,5nC2BQzszGGkZi7cvxFzH4,spotify:track:5nC2BQzszGGkZi7cvxFzH4,https://api.spotify.com/v1/tracks/5nC2BQzszGGk...,https://api.spotify.com/v1/audio-analysis/5nC2...,225427,4
114,Family of the Year,Hero,Loma Vista,0.341,0.539,5,-5.760,1,0.0265,0.2130,...,0.1090,0.267,174.564,audio_features,6GRDI9suQHikFP6euIXnpq,spotify:track:6GRDI9suQHikFP6euIXnpq,https://api.spotify.com/v1/tracks/6GRDI9suQHik...,https://api.spotify.com/v1/audio-analysis/6GRD...,190280,4
